## Data preprocess

`preprocess.py`
combine all dialogs into one text file, one dialog one line, retaining conversation only, removing punctuation, removing stopword

`word_count.py`
do the word count

## notes in preprocessing
### stop word
I don't have a good stopword resource regarding system dialog, so I first do the word count with common stopword list, then manually add some extra sotp words

btw, filtering out stop word might be time consuming using Regex, a good tool for this is [flashtext](https://github.com/vi3k6i5/flashtext), a word matching model based on prefix-tree. Its performance remain O(n), where n is the length of sentence, while Regex might go up to O(mn) where m is the stopword size, which means the flashtext is performance-insentitive regarding the size of keyword set. It reduce my processing time significantly. 
### stemming
Stemming means combining similar word into one. For example, **install** and **installs**

Here I use the Stemmer from [NLTK](http://www.nltk.org/)
But I found that the stemmer might trim the word at some wrong position. But it's fined to construct word-vector space.

### tokenize
basically you can write a simple Regex to do this, or even string.split() will do. But I to gain better performance, I use tokenizer from NLTK too. The regex I use for tkenizer is **r'\w\S+\w'**, which retain a word start with character and end with character, this can filter out punctuation, but keep sth. like **don't**  or **pre-installed**, where punctuation appear inside a word.

In [ ]:
# %load preprocess.py

'''

@author: ZiqiLiu


@file: preprocess.py

@time: 2017/11/11 上午1:56

@desc: combine all dialogs into one text file, retaining dialogs only
'''
from glob import glob
from tqdm import tqdm
import codecs
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from stop_words import get_stop_words
from flashtext import KeywordProcessor
import pickle
import re
import os
from multiprocessing import Pool
import functools

files = glob('./dialogs/**/*.tsv')


# pattern1 = re.compile(r'[1-9\\.]+')
# pattern2 = re.compile(r'.+//.+')
# pattern3 = re.compile(r'\w+')
#
#
# def filter1(s):
#     return pattern1.match(s) and pattern2.match(s) and pattern3.match(s)



def processing(i, file_list):
    tk = RegexpTokenizer(r'\w\S+\w')

    # create English stop words list
    en_stop = get_stop_words('en')

    stopword_processor = KeywordProcessor()
    for w in en_stop:
        stopword_processor.add_keyword(w, ' ')

    with open('stopword_processor.pkl', 'wb') as f:
        pickle.dump(stopword_processor, f)

    p_stemmer = PorterStemmer()

    with codecs.open('whole_dialogs_stem_%d' % i, 'w', 'utf-8') as out:
        for fi in tqdm(file_list):
            with codecs.open(fi, 'r', 'utf-8') as f:
                sentences = [stopword_processor.replace_keywords(
                    line.strip().split('\t')[-1].lower()) for
                    line in f]
                words = functools.reduce(lambda x, y: x + y,
                                         map(tk.tokenize, sentences))
                words = map(p_stemmer.stem, words)
                out.write(' '.join(words) + '\n')


def div_list(l, n):
    length = len(l)
    t = length // n
    quaters = [t * i for i in range(0, n)]
    ran = range(0, n - 1)
    result = [l[quaters[i]:quaters[i + 1]] for i in ran]
    result.append(l[quaters[n - 1]:len(l)])
    return result


if __name__ == '__main__':
    process_num = 8
    p = Pool()
    div_files = div_list(files, process_num)
    for i in range(process_num):
        p.apply_async(processing, args=(
            i, div_files[i]))
    p.close()
    p.join()

    output_list = glob('./whole_dialogs_stem_*')
    for output in output_list:
        os.system('cat %s >> whole_dialogs_stem' % output)
        os.system('rm %s' % output)


In [ ]:
# %load word_count.py

'''

@author: ZiqiLiu


@file: word_count.py

@time: 2017/11/11 上午2:58

@desc:
'''

import codecs
from collections import Counter
import pickle
from tqdm import tqdm
import re
from flashtext import KeywordProcessor
from my_prefix_tree import PrefixTree

with open('stopword_for_wc', 'r') as f:
    stop_word_addon = [line.strip() for line in f]

split_pattern = re.compile(r'\s+')

word_counter = Counter()

with codecs.open('whole_dialogs', 'r', 'utf-8') as f:
    for line in tqdm(f):
        word_list = split_pattern.split(line)
        word_counter.update(word_list)
for stop_w in stop_word_addon:
    del word_counter[stop_w]
top_words = sorted(word_counter.items(), key=lambda a: a[1], reverse=True)[:5000]

with codecs.open('wc', 'w', 'utf-8') as out:
    for key, value in top_words:
        out.write('%s\t%d\n' % (key, value))

keyword_processer = PrefixTree()
for w in top_words:
    keyword_processer.update(w[0])

with open('keyword_processor.pkl', 'wb') as f:
    pickle.dump(keyword_processer, f)


### Ok, let's have a look at top 100 words (doesn't use stemmer here because might result in wrongly truncated word)

In [1]:
!cat wc | head -n100

ubuntu	1043670
install	769511
sudo	421245
file	388421
windows	373818
installed	339438
linux	312921
make	268000
boot	258460
system	245784
apt-get	227627
server	209155
files	204674
package	199069
partition	192931
drive	182006
terminal	174528
version	173138
root	164448
desktop	157562
kernel	156742
gnome	154771
driver	153281
mount	139047
drivers	136361
network	126579
usb	120563
update	114767
remove	111887
packages	106975
installing	105368
download	102518
firefox	101489
nvidia	100199
directory	99513
upgrade	99092
disk	97785
manager	96632
channel	95727
wireless	94553
option	91186
software	88775
password	87301
video	86580
folder	84099
etc	83512
synaptic	83296
link	82586
menu	81315
device	80534
list	80483
connect	75136
flash	75055
source	74444
ssh	73894
gui	73731
google	70793
login	69963
kde	68881
settings	68830
window	68124
internet	65119
reboot	64409
log	63608
iso	61454
setup	61123
mode	61021
connection	60183
reinstall	59991
debian	59383
ram	59292
ati	58391
configure	58297
pastebin	57037
com

## topic detection
here I come up with two possible solution.

### 1. Naive pick the key words in the dialog. 

The hard thing is it's sometimes diffucult to distinguish stopword and keyword. By looking at frequency might get a little help, as most stopword apprear in high frequency. But keyword like 'linux' or 'windows' also have high frequency. So generally speaking, if we don't have a good 'keyword list' and 'stop word list', it's hard to do so. And the keyword list and stopword list might need to be somehow manually constructed, or using more advanced NLP model.

The approach I use here, is that I manually pick up hundreds of keywords with highest frequency, and simply extract them from dialogs.

I manually build a prefix-tree based on word count. It allow partial match: for example, **ubuntu-server** can match **ubuntu** 



### 2. LDA (Latent Dirichlet allocation)

Basically it's a unsupervised learning model based on crazy statistic. It assumes a article is generated from a mix of  hidden topics, and each model contains a potential words that can be drawn from.

So literally we have two distribution, the topic model distribution p(topic|θ) and a word distrubition given a topic model p(word|topic). Notice that θ is a hyper-pamameter

Then to generate a article, the process is here:

* Chooseparameter θ ～ p(θ); 
* For each of the N words:   
* Choose a topic ～ p(topic|θ); 
* Choose a word ～ p(word|topic);
                
                
And to classify the topic given an article, we choose the topic that has the highest probability to generate this article. It's similar to maximum posterior probability.

But the problem here is it's a unsupervised learning, and actually we don't have a 'keyword' for a topic, what we use to represent the feature of a topic is a word-vector, which is a distribution of word given this topic. Of course we can choose some high frequency words as the key word in this model. 

And another problem is that, it's likely that given a dialog, we might suggest a keyword not in the article! This might look wried.

But on the other hand, this is a good thing, as this model consider the hidden semantic structure. For example, a simple dialog ** Hey, I failed to use 'sudo apt install python3'**. A keyword **python3** can be easily detected, but a human being with some linux knowledge should be able to tell it's a ubuntu-like distribution. And the LDA model might be able to do this.

And here is the possible situation:

User input: ** Hey, I failed to use 'sudo apt install python3'**

Response: your problem might be relative to this topics: **python3, ubuntu, apt, install**

And the word **'ubuntu'** looks reasonable even if it doesn't appear in the dialog.


### let's tried the naive model first

usage:
`python3 predict_naive.py <*.tsv>..`
following arguments can be any number of ubuntu dialog in raw tsv format

In [8]:
!python3 predict_naive.py ./test/100001.tsv ./test/100002.tsv ./test/100003.tsv

++++++++++++++++++++++++++++++++++++++++++++++++++
file:	./test/100001.tsv
------------------------------
raw dialog:
2008-09-02T04:02:00.000Z	chuck	brettley	HAI THAR
2008-09-02T04:04:00.000Z	brettley	chuck	go awai
2008-09-02T04:10:00.000Z	chuck	brettley	hello


------------------------------
keywords:
[]
++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++
file:	./test/100002.tsv
------------------------------
raw dialog:
2008-09-03T06:57:00.000Z	brettley		if i installed another linux opperating system next to ubuntu, would it take the GRUB i have now and modify it?
2008-09-03T06:59:00.000Z	wols_	brettley	it would install anotehr bootloader
2008-09-03T07:00:00.000Z	brettley	wols_	like what?


------------------------------
keywords:
['installed', 'linux', 'system', 'ubuntu', 'install', 'bootloader']
++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++
file:	./test/100003.tsv
-------------

#### looks reasonable

### And let's train a LDA model

In [ ]:
# %load LDA.py

'''

@author: ZiqiLiu


@file: LDA.py

@time: 2017/11/11 下午1:34

@desc:
'''

from gensim import corpora, models
import codecs
from tqdm import tqdm

print('loading data...')
with codecs.open('whole_dialogs_stem', 'r', 'utf-8') as f:
    raw_corpus = [line.split() for line in f]
print("%d dialogs loaded." % len(raw_corpus))

print('begin training...')
dictionary = corpora.Dictionary(raw_corpus)
dictionary.filter_extremes(keep_n=200000)

corpus = [dictionary.doc2bow(text) for text in raw_corpus]
model = models.LdaMulticore(corpus, id2word=dictionary, workers=3,
                            num_topics=50)

dictionary.save('ubuntu.dict')
model.save('ubuntu.lda')

print(
    'training finished. Model saved in ubuntu.lda. Dictionary saved in ubuntu.dic')


### Ok, let's see how LDA perform.

In [18]:
!python3 predict_LDA.py ./test/100001.tsv ./test/100002.tsv ./test/100003.tsv

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

file:	./test/100001.tsv
--------------------------------------------------
raw dialog:
2008-09-02T04:02:00.000Z	chuck	brettley	HAI THAR
2008-09-02T04:04:00.000Z	brettley	chuck	go awai
2008-09-02T04:10:00.000Z	chuck	brettley	hello


--------------------------------------------------
LDA predict:

potential topic
[(15, 0.50454097205436121), (24, 0.25545902794563885)]

potential keywords:
0.216*"ubuntu" + 0.051*"debian" + 0.043*"use" + 0.038*"distro" + 0.026*"support" + 0.022*"base" + 0.017*"develop" + 0.015*"gentoo" + 0.013*"fedora" + 0.011*"like"

filtered keywords:
set()

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

file:	./test/100002.tsv
--------------------------------------------------
raw dialog:
2008-09-03T06:57:00.000Z	brettley		if i installed another linux opperating system next to

### the extracted keywords is given by "filtered keywords"

Seems reasonable.

### some discussion here:

1. First test case give null keyword, which is reasonable, as that dialog is meaningless.  But if we don't filter the keywords (i.e, only keep those keywords appear in topic words), the first test case, which only few meaningless keyword, will give strange results, as its still forced classified into some category, which the dialog is meaningless at all.

2. Actually I tried to train the LDA model based on un-stemming corpus, which totally mess up. Prove that saysing: garbage in garbage out. Stemming is a critical preprocessing to construct a clear word vector space, and it's critical to those models based on word vector space and statistic&probability.


## conclusion

1. In the experiment, the naive method can give reasonable keyword regarding high frequency topic.
Potential imporvement: bigram and trigram

2. But the LDA also give reasonable result, but it sometime mess up with part of the test case. Aside from the discussion above, I still have some further thoughts: 
    * need further preprocessing. In ubuntu dialog corpus, there're many file path, URL, numbers, either filtered these out, or mapping them to specific word vector.
    * need parameter tuning. Due to the time and computing resource limit I haven't tune the paramter very well, just pick reasonable and conservative number.
    * The dialog is quite short, with only few word that can represent the feature of the topic. I have a project using LDA for topic classification of newspaper articles, which make much sense than ubuntu corpus. I suspect whether LDA is a good model for such a corpus.There're still many statistic and probability based model to try.

